In [1]:
import ampcor
from matplotlib import matplotlib as 

In [2]:
# make an execution environment
app = ampcor.shells.ampcor(name='plexus')
# configure its layout
app.shell.hosts = 1
app.shell.tasks = 1
app.shell.gpus = 1

In [3]:
# workspace management
ws = ampcor.executive.fileserver
# mount the data directory
ws["BDL"] = ampcor.filesystem.local(root="/data/BDL").discover()
# create the output directory
# ws.mkdir(name="ampcor_output",parent="BDL")
BDL=ws["BDL"]
outputdir=BDL.mkdir(name="ampcor_output",exist_ok=True)
# realize the output file
offout=BDL.touch(name="ampcor_output/testoffsets.bin")
# show me the contents

In [4]:
# data products
# the reference SLC
ref = ampcor.products.newSLC(name="slcref")
# its shape
ref.shape = 3241,4321
# the path to the data file
ref.data = f"/BDL/{ref.pyre_name}"


In [7]:
# the secondary SLC
sec = ampcor.products.newSLC(name="slcsec")
# its shape
sec.shape = 3241,4321
# the path to the data file
sec.data = f"/BDL/{sec.pyre_name}"



In [8]:
# the output
single = ampcor.products.newOffsets(name="single")
# the plan shape; determines how many correlation pairs will be computed
single.shape = 1,1
# the path to the output file
single.data = offout.uri 


In [9]:
# the generator of points on the reference image
# {uniformGrid} is the default, so this is not strictly speaking necessary
# it doesn't have any user configurable state either
uniform = ampcor.correlators.newUniformGrid(name="uni")


In [10]:
# the functor that maps points in the reference image onto the secondary image
# used by the tile pair generator below to place tiles in the secondary image
constant = ampcor.correlators.newConstant(name="zero")
# you can use this to apply a constant shift
constant.shift = 0,0

In [13]:
# these two together become the strategy for pairing up points in the reference image
# to points in the secondary image to form tiles
grid = ampcor.correlators.newGrid(name="griddy")
# the strategy for generating points in the reference image
grid.domain = uniform
# and mapping these onto points in the secondary image
grid.functor = constant

In [14]:
# the correlator
mga = ampcor.correlators.newMGA(name="large")
# the inputs
mga.reference = ref
mga.secondary = sec
# the output
mga.offsets = single
# the generator of pairs of points in the reference and secondary images
mga.cover = grid
# configuration
# reference chip size
mga.chip = 128, 128
# padding used to form the search window in the secondary raster
mga.padding = 32, 32
# the refinement factor
mga.refineFactor = 4
# the refinement margin
mga.refineMargin = 8
# the zoom factor
mga.zoomFactor = 8
# show me
print("\n".join(mga.show(indent=" "*2, margin="")))

estimator:
  name: large
  family: ampcor.correlators.mga
  chip: (128, 128)
  padding: (32, 32)
  refinement factor: 4
  refinement margin: 8
  zoom factor: 8
  initial cover generator:
    name: griddy
    family: ampcor.correlators.covers.grid
    domain:
      name: uni
      family: ampcor.correlators.domains.uniform
    functor: ampcor.correlators.functors.constant
      name: zero
      family: ampcor.correlators.functors.constant
      shift: (0, 0)
  plan:
    shape: (1, 1), layout: (1, 0)
    pairs: 1 out of 1
    footprint:
      reference: 16384 cells in 131072 bytes
      secondary: 36864 cells in 294912 bytes
    pair: (0, 0)
      ref:
        origin: (1556, 2096)
        shape: (128, 128)
      sec:
        origin: (1524, 2064)
        shape: (192, 192)


In [15]:
# invoke the correlator
mga.estimate(plexus=app)

0